In [ ]:
pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import ray

# Define the CNN model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define a remote function to train the model on a worker node
@ray.remote
def train_on_worker(x_train, y_train, epochs):
    model = create_model()
    model.fit(x_train, y_train, epochs=epochs, verbose=0)
    weights = model.get_weights()
    return weights

# Initialize Ray
ray.init()

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Split the data into 3 parts and distribute to the worker nodes
x_splits = np.array_split(x_train, 3)
y_splits = np.array_split(y_train, 3)
worker_returns = []

# Train the model on each worker node
for i in range(3):
    worker_returns.append(train_on_worker.remote(x_splits[i], y_splits[i], epochs=5))

# Get the weights from each worker node
weights = ray.get(worker_returns)

# Combine the weights from each worker node
new_weights = []
for i in range(len(weights[0])):
    w = []
    for j in range(len(weights)):
        w.append(weights[j][i])
    new_weights.append(np.mean(w, axis=0))

# Set the weights of the model to the new combined weights
model = create_model()
model.set_weights(new_weights)

# Evaluate the model on the test set
_, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {accuracy}')


2023-03-28 19:21:55,658	INFO worker.py:1553 -- Started a local Ray instance.
(pid=56842) 2023-03-28 19:21:58.326641: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=56842) 2023-03-28 19:21:58.326730: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=56842) 2023-03-28 19:21:58.326744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
(pid=56844) 2023-03-28 19:21:58.328182: W tensorflow/c

Test accuracy: 0.11190000176429749
